# Regression

This notebook compares regression models from various libraries. The dataset used is the [Kaggle Recruit Restaurants dataset](https://creme-ml.github.io/generated/creme.datasets.fetch_restaurants.html#creme.datasets.fetch_restaurants).

In [2]:
%load_ext watermark
%watermark --python --machine --packages creme,keras,skgarden,sklearn,tensorflow,torch --datename

Using TensorFlow backend.


Wed Aug 21 2019 

CPython 3.7.3
IPython 7.4.0

creme 0.3.0
keras 2.2.4
skgarden 0.1.2
sklearn 0.20.3
tensorflow 1.14.0
torch 1.1.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.1.9-arch1-1-ARCH
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit


In [3]:
from creme import compose
from creme import datasets
from creme import feature_extraction
from creme import linear_model
from creme import metrics
from creme import neighbors
from creme import optim
from creme import preprocessing
from creme import stats
from keras import layers
from keras import models
from keras import optimizers
import skgarden
from sklearn import linear_model as sk_linear_model
import torch

%run utils.py
%run wrappers.py

In [5]:
n_features = 5

class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.linear(x)
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1)(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(loss='mean_squared_error', optimizer='sgd')


def get_date_features(x):
    weekday =  x['date'].weekday()
    return {'weekday': weekday, 'is_weekend': weekday in (5, 6)}

results = benchmark(
    get_X_y=datasets.fetch_restaurants,
    n=252108,
    get_pp=lambda: (
        (
            compose.FuncTransformer(get_date_features) +
            feature_extraction.TargetAgg(by='store_id', how=stats.RollingMean(7)) +
            feature_extraction.TargetAgg(by='store_id', how=stats.RollingMean(14)) +
            feature_extraction.TargetAgg(by='store_id', how=stats.RollingMean(21)) 
        ) |
        compose.Blacklister('store_id', 'date', 'genre_name', 'area_name', 'latitude', 'longitude') |
        preprocessing.StandardScaler()
    ),
    models=[
        ('creme', 'LinearRegression', linear_model.LinearRegression()),
        ('creme', 'PARegressor', linear_model.PARegressor()),
        ('creme', 'KNeighborsRegressor', neighbors.KNeighborsRegressor()),
        ('scikit-learn', 'SGDRegressor', ScikitLearnRegressor(sk_linear_model.SGDRegressor())),
        ('scikit-learn', 'PassiveAggressiveRegressor', ScikitLearnRegressor(sk_linear_model.PassiveAggressiveRegressor())),
        ('PyTorch (CPU)', 'Linear', PyTorchRegressor(
            network=torch_model,
            loss_fn=torch.nn.MSELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=0.01)
        )),
        ('Keras on Tensorflow (CPU)', 'Dense', KerasRegressor(keras_model)),
        ('scikit-garden', 'MondrianTreeRegressor', ScikitLearnRegressor(skgarden.MondrianTreeRegressor(
            random_state=42
        ))),
        ('scikit-garden', 'MondrianForestRegressor', ScikitLearnRegressor(skgarden.MondrianForestRegressor(
            random_state=42
        )))
    ],
    get_metric=metrics.MSE
)

W0821 14:29:23.073089 140274460936000 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0821 14:29:23.074513 140274460936000 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [6]:
results

,Library,Model,MSE,Fit time,Average fit time,Predict time,Average predict time
0,creme,LinearRegression,118.549437,"2s, 580ms, 335μs, 806ns","10μs, 235ns","759ms, 277μs, 164ns","3μs, 12ns"
1,creme,PARegressor,143.477210,"6s, 994ms, 226μs, 455ns","27μs, 743ns","1s, 305ms, 952μs, 961ns","5μs, 180ns"
2,creme,KNeighborsRegressor,155.585250,"394ms, 893μs, 956ns","1μs, 566ns","37s, 4ms, 731μs, 422ns","146μs, 781ns"
3,scikit-learn,SGDRegressor,120.185848,"36s, 433ms, 335μs, 738ns","144μs, 515ns","14s, 766ms, 211μs, 576ns","58μs, 571ns"
4,scikit-learn,PassiveAggressiveRegressor,143.477210,"35s, 551ms, 211μs, 534ns","141μs, 16ns","14s, 599ms, 61μs, 497ns","57μs, 908ns"
5,PyTorch (CPU),Linear,142.495995,"47s, 335ms, 680μs, 555ns","187μs, 760ns","14s, 822ms, 474μs, 684ns","58μs, 794ns"
6,Keras on Tensorflow (CPU),Dense,142.494512,"1m, 18s, 296ms, 225μs, 913ns","310μs, 566ns","49s, 225ms, 980μs, 400ns","195μs, 258ns"
7,scikit-garden,MondrianTreeRegressor,201.687033,"35s, 983ms, 74μs, 12ns","142μs, 729ns","23s, 502ms, 259μs, 452ns","93μs, 223ns"
8,scikit-garden,MondrianForestRegressor,142.364156,"5m, 58s, 226ms, 553μs, 453ns","1ms, 420μs, 925ns","2m, 40s, 728ms, 575μs, 12ns","637μs, 539ns"


In [8]:
print(results.to_html(
    index=False,
    columns=['Library', 'Model', 'MSE', 'Fit time', 'Average fit time', 'Predict time', 'Average predict time'],
    border=0
))

<table border="0" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Library</th>
      <th>Model</th>
      <th>MSE</th>
      <th>Fit time</th>
      <th>Average fit time</th>
      <th>Predict time</th>
      <th>Average predict time</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>creme</td>
      <td>LinearRegression</td>
      <td>118.549437</td>
      <td>2s, 580ms, 335μs, 806ns</td>
      <td>10μs, 235ns</td>
      <td>759ms, 277μs, 164ns</td>
      <td>3μs, 12ns</td>
    </tr>
    <tr>
      <td>creme</td>
      <td>PARegressor</td>
      <td>143.477210</td>
      <td>6s, 994ms, 226μs, 455ns</td>
      <td>27μs, 743ns</td>
      <td>1s, 305ms, 952μs, 961ns</td>
      <td>5μs, 180ns</td>
    </tr>
    <tr>
      <td>creme</td>
      <td>KNeighborsRegressor</td>
      <td>155.585250</td>
      <td>394ms, 893μs, 956ns</td>
      <td>1μs, 566ns</td>
      <td>37s, 4ms, 731μs, 422ns</td>
      <td>146μs, 781ns</td>
    </tr>
    <tr>
      <td>scikit-l